In [1]:
import pandas as pd
import os
import requests



In [2]:
# Load the summary spreadsheet
spreadsheet_url = "https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Phenotypic_V1_0b.csv"
df = pd.read_csv(spreadsheet_url)



In [3]:
df

,SITE_ID,SUB_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,...,WISC_IV_BLK_DSN_SCALED,WISC_IV_PIC_CON_SCALED,WISC_IV_MATRIX_SCALED,WISC_IV_DIGIT_SPAN_SCALED,WISC_IV_LET_NUM_SCALED,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI
0,CALTECH,51456,Caltech_0051456,1,4,55.40,1,R,NaN,126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
1,CALTECH,51457,Caltech_0051457,1,4,22.90,1,Ambi,NaN,107.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
2,CALTECH,51458,Caltech_0051458,1,1,39.20,1,R,NaN,93.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
3,CALTECH,51459,Caltech_0051459,1,1,22.80,1,R,NaN,106.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
4,CALTECH,51460,Caltech_0051460,1,1,34.60,2,Ambi,NaN,133.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,YALE,50624,Yale_0050624,1,3,11.08,2,R,NaN,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
1108,YALE,50625,Yale_0050625,1,3,7.00,1,L,NaN,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
1109,YALE,50626,Yale_0050626,1,3,11.08,1,L,NaN,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
1110,YALE,50627,Yale_0050627,1,3,9.50,2,R,NaN,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [4]:
len(df["SUB_ID"].unique())

1112

In [5]:
df["SITE_ID"].unique()

array(['CALTECH', 'CMU', 'KKI', 'LEUVEN_1', 'LEUVEN_2', 'MAX_MUN', 'NYU',
       'OHSU', 'OLIN', 'PITT', 'SBL', 'SDSU', 'STANFORD', 'TRINITY',
       'UCLA_1', 'UCLA_2', 'UM_1', 'UM_2', 'USM', 'YALE'], dtype=object)

In [6]:
df["DX_GROUP"].unique()

array([1, 2])

In [7]:
df.columns

Index(['SITE_ID', 'SUB_ID', 'FILE_ID', 'DX_GROUP', 'DSM_IV_TR', 'AGE_AT_SCAN',
       'SEX', 'HANDEDNESS_CATEGORY', 'HANDEDNESS_SCORES', 'FIQ', 'VIQ', 'PIQ',
       'FIQ_TEST_TYPE', 'VIQ_TEST_TYPE', 'PIQ_TEST_TYPE',
       'ADI_R_SOCIAL_TOTAL_A', 'ADI_R_VERBAL_TOTAL_BV', 'ADI_RRB_TOTAL_C',
       'ADI_R_ONSET_TOTAL_D', 'ADI_R_RSRCH_RELIABLE', 'ADOS_MODULE',
       'ADOS_TOTAL', 'ADOS_COMM', 'ADOS_SOCIAL', 'ADOS_STEREO_BEHAV',
       'ADOS_RSRCH_RELIABLE', 'ADOS_GOTHAM_SOCAFFECT', 'ADOS_GOTHAM_RRB',
       'ADOS_GOTHAM_TOTAL', 'ADOS_GOTHAM_SEVERITY', 'SRS_VERSION',
       'SRS_RAW_TOTAL', 'SRS_AWARENESS', 'SRS_COGNITION', 'SRS_COMMUNICATION',
       'SRS_MOTIVATION', 'SRS_MANNERISMS', 'SCQ_TOTAL', 'AQ_TOTAL',
       'COMORBIDITY', 'CURRENT_MED_STATUS', 'MEDICATION_NAME',
       'OFF_STIMULANTS_AT_SCAN', 'VINELAND_RECEPTIVE_V_SCALED',
       'VINELAND_EXPRESSIVE_V_SCALED', 'VINELAND_WRITTEN_V_SCALED',
       'VINELAND_COMMUNICATION_STANDARD', 'VINELAND_PERSONAL_V_SCALED',
       'VINELAN

In [ ]:
# Select a subset of subjects (or filter based on criteria)
subject_ids = df["FILE_ID"].dropna().astype(str).tolist()

# Define parameters
pipeline = "cpac"
strategy = "filt_global"
derivative = "func_preproc"
# output_dir = "/blue/ruogu.fang/ryoi360/projects/fmri_vlm/data/ABIDE_fMRI"
output_dir = "/orange/ruogu.fang/ryoi360/ABIDE"


# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Download each fMRI file
for subject_id in subject_ids:  # Download only the first 10 subjects (modify as needed)
    file_url = f"https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/{pipeline}/{strategy}/{derivative}/{subject_id}_{derivative}.nii.gz"
    file_path = os.path.join(output_dir, f"{subject_id}_{derivative}.nii.gz")

    # Download file
    response = requests.get(file_url, stream=True)
    if response.status_code == 200:
        with open(file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded: {file_path}")
    else:
        print(f"Failed to download: {file_url}")

Downloaded: /orange/ruogu.fang/ryoi360/ABIDE/Caltech_0051456_func_preproc.nii.gz
Downloaded: /orange/ruogu.fang/ryoi360/ABIDE/Caltech_0051457_func_preproc.nii.gz
Downloaded: /orange/ruogu.fang/ryoi360/ABIDE/Caltech_0051458_func_preproc.nii.gz
Downloaded: /orange/ruogu.fang/ryoi360/ABIDE/Caltech_0051459_func_preproc.nii.gz
Downloaded: /orange/ruogu.fang/ryoi360/ABIDE/Caltech_0051460_func_preproc.nii.gz
Downloaded: /orange/ruogu.fang/ryoi360/ABIDE/Caltech_0051461_func_preproc.nii.gz
Downloaded: /orange/ruogu.fang/ryoi360/ABIDE/Caltech_0051462_func_preproc.nii.gz
Downloaded: /orange/ruogu.fang/ryoi360/ABIDE/Caltech_0051463_func_preproc.nii.gz
Downloaded: /orange/ruogu.fang/ryoi360/ABIDE/Caltech_0051464_func_preproc.nii.gz
Downloaded: /orange/ruogu.fang/ryoi360/ABIDE/Caltech_0051465_func_preproc.nii.gz
Downloaded: /orange/ruogu.fang/ryoi360/ABIDE/Caltech_0051466_func_preproc.nii.gz
Downloaded: /orange/ruogu.fang/ryoi360/ABIDE/Caltech_0051467_func_preproc.nii.gz
Downloaded: /orange/ruogu.fa